## Unzip The Raw Data

In [4]:
!./unzip.sh UCI_HAR_Dataset.zip 2>&1 > /dev/null

In [5]:
import tensorflow as tf
print(tf.__version__)

2.11.0


## Building The Dataset

In [6]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import L1L2
import numpy as np
import glob
import os

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

def load_y(subset):
    # Get the path
    path = f'UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/y_{subset}.txt'

    # Read the file
    y = np.loadtxt(path, delimiter=',', dtype=int)

    # # One-hot encode labels
    one_hot_labels = get_one_hot(y - 1 , len(np.unique(y)))
    if subset == 'train':
        assert one_hot_labels.shape == (7352, 6), f"Wrong dimensions: {one_hot_labels.shape} should be (7352, 6)"
    if subset == 'test':
        assert one_hot_labels.shape == (2947, 6), f"Wrong dimensions: {one_hot_labels.shape} should be (2947, 6)"
    assert y[0] - 1 == np.where(one_hot_labels[0] == np.max(one_hot_labels[0]))[0][0], f"Value mismatch {np.max(one_hot_labels[0])[0][0]} vs {y[13] - 1}"
    return one_hot_labels

def build_data(subset):
    if subset not in ['train', 'val', 'test']:
        raise Exception(f"Invalid subset: {subset}")

    folder_path = f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/"

    # Get all signal files in folder
    signal_files = glob.glob(os.path.join(folder_path, '*.txt'))
    # print(signal_files)

    assert len(signal_files) == 9, f"No signal files found in {folder_path}"
    signal_shape = np.loadtxt(signal_files[0]).shape
    # print(f"{signal_shape}")

    # Determine signal order based on file names
    signal_order = [
        "body_acc_x_",
        "body_acc_y_",
        "body_acc_z_",
        "body_gyro_x_",
        "body_gyro_y_",
        "body_gyro_z_",
        "total_acc_x_",
        "total_acc_y_",
        "total_acc_z_",
        ]

    # file_prefix = "UCI_HAR_Dataset/UCI_HAR_Dataset/train/Inertial Signals/"
    # file_suffix = ".txt"
    signal_files = [f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/{x}{subset}.txt" for x in signal_order]

    # Load signal data from each file and append to signals_data list
    signals_data = [np.loadtxt(x) for x in signal_files]

    # Transpose signal data array so that shape is (number of samples, number of timesteps, number of signals)
    signals_data = np.transpose(signals_data, (1, 2, 0))

    # Verify final shape of combined data
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    if subset == 'train':
        assert signals_data.shape == (7352, 128, len(signal_files))
    else:
        assert signals_data.shape == (2947, 128, len(signal_files))
    return signals_data

def load_data():
    return build_data('train'), load_y('train'), build_data('test'), load_y('test')

# Loading the train and test data
X_train, y_train, X_test, y_test = load_data()

In [7]:
first_sample = X_train[0]
first_timestep = first_sample[0]
assert len(first_sample) == 128
assert first_timestep[0] == 1.8085150e-004, print(first_timestep[0])
assert first_timestep[1] == 1.0766810e-002, print(first_timestep[1])

In [8]:
#function to count the number of classes
def count_classes(y):
    return len(set([tuple(category) for category in y]))

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = count_classes(y_train)

# Initializing parameters
n_epochs = 30
n_batch = 16

# Bias regularizer value - we will use elasticnet
regularizer = L1L2(0.01, 0.01)

print(f"Timesteps: {timesteps}")
print(f"Input dimention: {input_dim}")
print(f"Total samples: {len(X_train)}")

Timesteps: 128
Input dimention: 9
Total samples: 7352


In [9]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=regularizer))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

2023-04-17 04:46:43.228332: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 04:46:43.228627: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-17 04:46:43.228786: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-17 04:46:43.228834: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-17 04:46:43.228978: W tensorflow/c

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 64)           18944     
                                                                 
 batch_normalization (BatchN  (None, 128, 64)          256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 48)                21696     
                                                                 
 dropout_1 (Dropout)         (None, 48)                0         
                                                                 
 dense (Dense)               (None, 6)                 294       
                                                        

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Epoch 1/30


2023-04-17 04:46:44.141430: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 33878016 exceeds 10% of free system memory.


460/460 [==============================] - 27s 53ms/step - loss: 1.6630 - accuracy: 0.7184 - val_loss: 1.0917 - val_accuracy: 0.8249
Epoch 2/30
460/460 [==============================] - 23s 51ms/step - loss: 0.8161 - accuracy: 0.8751 - val_loss: 0.6250 - val_accuracy: 0.8839
Epoch 3/30
460/460 [==============================] - 24s 51ms/step - loss: 0.4051 - accuracy: 0.9212 - val_loss: 0.4531 - val_accuracy: 0.8656
Epoch 4/30
460/460 [==============================] - 24s 52ms/step - loss: 0.2207 - accuracy: 0.9332 - val_loss: 0.3624 - val_accuracy: 0.8812
Epoch 5/30
460/460 [==============================] - 24s 53ms/step - loss: 0.1836 - accuracy: 0.9342 - val_loss: 0.2159 - val_accuracy: 0.9226
Epoch 6/30
460/460 [==============================] - 24s 53ms/step - loss: 0.1523 - accuracy: 0.9407 - val_loss: 0.2678 - val_accuracy: 0.9050
Epoch 7/30
460/460 [==============================] - 24s 53ms/step - loss: 0.1530 - accuracy: 0.9422 - val_loss: 0.2692 - val_accuracy: 0.9240
Epo

In [12]:
# Save model(s) for development purposes
import tensorflow as tf

# create a TFLiteConverter object
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# set the target ops to TFLITE_BUILTINS and SELECT_TF_OPS
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# disable lowering tensor list operations
converter._experimental_lower_tensor_list_ops = False

# convert the model to TFLite format
tflite_model = converter.convert()

# save the TFLite model to a file
with open('./assets/model_lstm.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp3zmbhavp/assets


INFO:tensorflow:Assets written to: /tmp/tmp3zmbhavp/assets
2023-04-17 05:10:21.614991: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-17 05:10:21.615026: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-04-17 05:10:21.616477: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp3zmbhavp
2023-04-17 05:10:21.625871: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-17 05:10:21.626142: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp3zmbhavp
2023-04-17 05:10:21.704403: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-04-17 05:10:21.849580: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp3zmbhavp
2023-04-17 05:10:21.936875: I tensorflow/cc/saved_model/loader.cc:305] SavedModel